# Домашнее задание
Описание/Пошаговая инструкция выполнения домашнего задания:

1.Возьмите ОДНО задачи с бенчмарка Russian SuperGLUE (https://russiansuperglue.com/tasks/)

2.Зафайнтьюньте для него модель c сайта huggingface (например, модель RuBERT)

3.Сделайте самбит на Leaderboard https://russiansuperglue.com/leaderboard/2 для оценки качества (в ноутбуке прикрепите скрин сабмита с результатами)

4.Создайте телеграм-бота, в котором будет поднята эта модель. Бот должен вначале писать вводное сообщение с описанием задачи, обрабатывать запрос пользоваля и выводить ответ для заданного пользователем примера

Замечание: пользователь должен писать боту запрос с примером из соотвествующего задания RussianSuperGLUE. под которого зафайнтьюнена модель. Возможны разные варианты реализации: пользователь пишет пример со всеми полями в одном сообщении или в нескольких. Конкретная реализация за вами.

Замечание: для проверки можете отправить код с телеграм-ботом и приложить скрины с примерами его работы (в случае если нет возможности поддерживать его в поднятом состоянии)

# Часть 1. Fine-tuning ru-BERT

In [ ]:
!pip install transformers

In [ ]:
import requests
import numpy as np
import pandas as pd
import time
import random
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import json
import random
from tqdm import tqdm
from sklearn.metrics import *

random.seed(42)
np.random.seed(42)
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# https://russiansuperglue.com/tasks/
# Yes/no Question Answering Dataset for the Russian

!wget --no-check-certificate https://russiansuperglue.com/tasks/download/DaNetQA

--2024-09-21 08:39:36--  https://russiansuperglue.com/tasks/download/DaNetQA
Resolving russiansuperglue.com (russiansuperglue.com)... 149.100.138.62
Connecting to russiansuperglue.com (russiansuperglue.com)|149.100.138.62|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1293761 (1.2M) [application/zip]
Saving to: ‘DaNetQA’

DaNetQA             100%[===================>]   1.23M  1.41MB/s    in 0.9s    

2024-09-21 08:39:38 (1.41 MB/s) - ‘DaNetQA’ saved [1293761/1293761]



In [ ]:
!unzip DaNetQA -d QA

Archive:  DaNetQA
   creating: QA/DaNetQA/
  inflating: QA/DaNetQA/train.jsonl  
   creating: QA/__MACOSX/
   creating: QA/__MACOSX/DaNetQA/
  inflating: QA/__MACOSX/DaNetQA/._train.jsonl  
  inflating: QA/DaNetQA/.DS_Store    
  inflating: QA/__MACOSX/DaNetQA/._.DS_Store  
  inflating: QA/DaNetQA/test.jsonl   
  inflating: QA/__MACOSX/DaNetQA/._test.jsonl  
  inflating: QA/DaNetQA/val.jsonl    
  inflating: QA/__MACOSX/DaNetQA/._val.jsonl  
  inflating: QA/__MACOSX/._DaNetQA   


In [ ]:
train = pd.read_json('/content/QA/DaNetQA/train.jsonl', orient='records', lines = True)
val = pd.read_json('/content/QA/DaNetQA/val.jsonl', orient='records', lines = True)
test = pd.read_json('/content/QA/DaNetQA/test.jsonl', orient='records', lines = True)

In [ ]:
"""
Вопрос:почему при использовании относительного пути, который начинает с текущей директории train = pd.read_json('/content/QA/DaNetQA/train.jsonl'),
в которой выполняется код, файлы локально сохраняются, а при использовании стандартного пути к
директории Google Drive в Google Colab файл не видит указанную папку (директорию)

train = pd.read_json('/content/drive/My Drive/DaNetQA/train.jsonl', orient='records', lines = True)
val = pd.read_json('/content/drive/My Drive/DaNetQA/val.jsonl', orient='records', lines = True)
test = pd.read_json('/content/drive/My Drive/DaNetQA/test.jsonl', orient='records', lines = True)
"""

ValueError: Expected object or value

In [ ]:
# Проверка наличия файлов
!ls "/content/drive/My Drive/DaNetQA/"

ls: cannot access '/content/drive/My Drive/DaNetQA/': No such file or directory


In [ ]:
train.head()

,question,passage,label,idx
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True,0
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True,1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True,2
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True,3
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False,4


In [ ]:
# Разделяем два предложения специальным символом [SEP]
from collections import Counter

train_X = train['question'] + '[SEP]' + train['passage']
train_y = train['label'].astype(int)
val_X = val['question'] + '[SEP]' + val['passage']
val_y = val['label'].astype(int)
test_X = test['question'] + '[SEP]' + test['passage']

print('Train size:', len(train_X))
print('Val size:', len(val_X))
print('Test size:', len(test_X))
print('\n')
print('Train labels counts\n', Counter(train_y), '\n')
print('Eval labels counts\n', Counter(val_y), '\n')

Train size: 1749
Val size: 821
Test size: 805


Train labels counts
 Counter({1: 1061, 0: 688}) 

Eval labels counts
 Counter({1: 412, 0: 409}) 



## Обучение Train Loop

In [ ]:
class TrainDataset(Dataset):

    def __init__(self, X, label, max_len):
        self.text = X.reset_index(drop=True)
        self.label = label.reset_index(drop=True)
        self.max_len = max_len

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len)

    def __len__(self):
        return self.label.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        output.update({'labels': torch.tensor(self.label[index])})
        return {k: v.reshape(-1).to(device) for k, v in output.items()}

class TestDataset(Dataset):

    def __init__(self, X, max_len):
        self.text = X.reset_index(drop=True)
        self.max_len = max_len

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1).to(device) for k, v in output.items()}

In [ ]:
def train_model(train_dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} / {num_epochs} \n -------------------')
        for n_batch, batch in enumerate(train_dataloader):
            # Forward pass (скормим данные в нейросеть и пробросим вперед)
            outputs = model(**batch)
            loss = outputs.loss
            if n_batch % 50 == 0:
                loss_value, current = loss.item(), n_batch * batch['input_ids'].shape[0]
                print(f"Loss train: {loss_value:>7f}  [{current:>5d}/{len(train_ds):>5d}]")
                print('Evaluating...')
                preds, true = test_model(eval_dataloader, eval=True)
                print(f'F1-score = {f1_score(preds, true, average="macro"):>3f}\n')
            # Посчитаем градиенты по всем параметрам с помощью обратного распространения ошибки)
            loss.backward()
            # Обновим параметры с помощью optimizer.step()
            optimizer.step()
            lr_scheduler.step()
            # Обнулим градиенты предыдущего шага
            optimizer.zero_grad()

def test_model(test_dataloader, eval=False):
    model.eval()
    y_pred = np.array([])
    y_true = np.array([])
    for n_batch, batch in enumerate(test_dataloader):
        if eval:
            y_true = np.hstack([y_true, batch['labels'].cpu().numpy().reshape(-1)])
        outputs = model(**batch)
        y_pred = np.hstack([y_pred, outputs['logits'].argmax(axis=1).detach().cpu().numpy()])
    return y_pred, y_true

In [ ]:
def get_reply(text1, text2):
  with torch.no_grad():
    model.eval()
    input_msg = tokenizer(text1, text2, return_tensors='pt', padding='max_length', truncation=True, max_length=max_len)
    input_msg = input_msg.to(device)
    outputs = model(**input_msg)[0]
    output = outputs.argmax(axis=1).detach().cpu().numpy()
    probabilities = torch.softmax(outputs, dim=-1).detach().cpu().numpy()
    print(text1, output, probabilities.max())


## Модель ruBert

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base")
# Copy the model to the GPU.
model.to(device)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# Считаем максимальное количество токенов в предложении
max_len = 0

# For every sentence...
for sent in train_X:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  751


In [ ]:
# Max sentence length = 751, превышает допутимую для ruBert, поэтому выставляем максимальную для ruBert
max_len = 512

In [ ]:
batch_size = 4

train_ds = TrainDataset(pd.Series(train_X), pd.Series(train_y), max_len)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

eval_ds = TrainDataset(pd.Series(val_X), pd.Series(val_y), max_len)
eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, shuffle=True)

test_ds = TestDataset(pd.Series(test_X), max_len)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

In [ ]:
# Задаем optimizer и sheduler, которые помогут с fine-tuning.

optimizer = Adam(model.parameters(), lr=5e-6)

num_epochs = 3
total_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps
)

In [ ]:
train_model(train_dataloader, num_epochs)

Epoch 1 / 3 
 -------------------
Loss train: 0.568743  [    0/ 1749]
Evaluating...
F1-score = 0.434149

Loss train: 0.470024  [  200/ 1749]
Evaluating...
F1-score = 0.334144

Loss train: 0.694226  [  400/ 1749]
Evaluating...
F1-score = 0.334144

Loss train: 0.481109  [  600/ 1749]
Evaluating...
F1-score = 0.352297

Loss train: 0.601572  [  800/ 1749]
Evaluating...
F1-score = 0.376740

Loss train: 0.716697  [ 1000/ 1749]
Evaluating...
F1-score = 0.534479

Loss train: 0.731863  [ 1200/ 1749]
Evaluating...
F1-score = 0.477480

Loss train: 0.546882  [ 1400/ 1749]
Evaluating...
F1-score = 0.535422

Loss train: 0.384453  [ 1600/ 1749]
Evaluating...
F1-score = 0.479312

Epoch 2 / 3 
 -------------------
Loss train: 0.688096  [    0/ 1749]
Evaluating...
F1-score = 0.388771

Loss train: 0.306755  [  200/ 1749]
Evaluating...
F1-score = 0.513288

Loss train: 0.348551  [  400/ 1749]
Evaluating...
F1-score = 0.484624

Loss train: 0.403351  [  600/ 1749]
Evaluating...
F1-score = 0.468902

Loss trai

In [ ]:
# Сохраняем модель
import os

output_dir = '/content/drive/MyDrive/ruBert-DaNetQA/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/ruBert-DaNetQA/


ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [ ]:
# Второй вариант сохранения ??? а-ля через trainer
# Сохранение модели с помощью `Train_model`
model_save_path = "/content/drive/MyDrive/HW5/ruBert-DaNetQA"
train_model.save_model(model_save_path)

# Сохранение конфигурации модели с помощью `Trainer.save_config`
train_model.save_config(model_save_path/config.json)

ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [ ]:
# Тестирование модели на данных, загруженных из `test_dataloader и возвращает предсказания (`y_pred`).
y_pred, _ = test_model(test_dataloader, eval=False)

In [ ]:
# Формирование выходного списка:
output = ["true" if i == 1 else "false"  for i in y_pred]
output = [f'{{"idx": {n}, "label": "{i}"}}' for n, i in enumerate(output)]

In [ ]:
# Запись в файл:
with open('/content/drive/MyDrive/DaNetQA.jsonl', 'w') as f:
    f.writelines('\n'.join(output))

In [ ]:
# Загрузить модели
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/ruBert-DaNetQA")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/ruBert-DaNetQA")

# Copy the model to the GPU.
model.to(device)